In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib 
import matplotlib.pyplot as plt 
import seaborn as sns 
import missingno as msno
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
train_data = pd.read_csv('used_car_train_20200313.zip', sep=' ')
test_data = pd.read_csv('used_car_testB_20200421.zip', sep=' ')

print(f'train data shape: {train_data.shape}')
print(f'test data shape: {test_data.shape}')

## 1. Exploratory Data Analysis
- I referenced the notebook in Tianchi, ['Task2'](https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12586969.1002.3.1cd837c9giLdfj&postId=95457)

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.describe()

**feature type and missing value check**
- features:'model', 'bodyType', 'fuelType' and 'gearbox', have some missing values.
- we may impute the missing values or leave them for tree-based models as they can learn the missing-value patterns.

In [ ]:
train_data.info()

In [ ]:
_ = msno.matrix(train_data.sample(1000),figsize=(15, 6))

In [ ]:
_ = msno.matrix(test_data.sample(1000), figsize=(15, 6))

In [ ]:
train_data.nunique()

In [ ]:
# process the notRepariedDamage feature which has unreasonable value, and mark it down as 'NaN' value
train_data.notRepairedDamage.value_counts()

In [ ]:
train_data['notRepairedDamage'].replace('-', np.nan, inplace=True)

In [ ]:
test_data.notRepairedDamage.value_counts()

In [ ]:
test_data['notRepairedDamage'].replace('-', np.nan, inplace=True)

In [ ]:
# two features, 'seller' and 'offerType', are useless
# and they will be deleted in the modeling stage
train_data.seller.value_counts()

In [ ]:
train_data.offerType.value_counts()

**analyze numerical and categorical features**

***1. Numerical features***
  - the price has strong correlation with kilometer, v_0, v_3, v_8 and v_12
  - some features are also highly correlated, e.g. v_3 and v_12, such that we may only keep either one as input into the models.

In [ ]:
numeric_features = ['power', 'kilometer', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13','v_14' ]

categorical_features = ['name', 'model', 'brand', 'bodyType', 'fuelType', 'gearbox', 'notRepairedDamage', 'regionCode']

In [ ]:
# 1. numerical features correlation analysis
correlation_numeric = train_data.loc[:, numeric_features + ['price']].corr()

f, ax = plt.subplots(figsize=(15, 15))
ax = sns.heatmap(correlation_numeric, square=True, annot=True)
ax.set_title('correlation among numerical features and target')

In [ ]:
# analyze the target variable, 'price'
# the target variable is highly left-skewed, such that some transformation may be needed.
ax = sns.distplot(train_data.price)

In [ ]:
# check the distribution for each numerical feature
# the 'power' variable has some values more than 600 which should be clipped
f = pd.melt(train_data, value_vars=numeric_features)
g = sns.FacetGrid(f, col="variable", col_wrap=6, sharex=False, sharey=False)
g = g.map(sns.distplot, "value")

***2.categorical features***

In [ ]:
train_data.loc[:, categorical_features].nunique()

In [ ]:
## plot the box_plot for some categorical features to see if they good predictors for the price target
categorical_features_plot =  ['brand',
 'bodyType',
 'fuelType',
 'gearbox']

plot_data = train_data.copy()
for c in categorical_features_plot:
    plot_data[c] = plot_data[c].astype('category')
    if plot_data[c].isnull().any():
        plot_data[c] = plot_data[c].cat.add_categories(['MISSING'])
        plot_data[c] = plot_data[c].fillna('MISSING')

def boxplot(x, y, **kwargs):
    sns.boxplot(x=x, y=y)
    x=plt.xticks(rotation=90)

f = pd.melt(plot_data, id_vars=['price'], value_vars=categorical_features_plot)
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, size=5)
g = g.map(boxplot, "value", "price")

## 2. Feature Engineering/Selection

In [ ]:
##2.1. creat new features for days and years the car owned.
train_data['used_days'] = (pd.to_datetime(train_data['creatDate'], format='%Y%m%d', errors='coerce') - 
                            pd.to_datetime(train_data['regDate'], format='%Y%m%d', errors='coerce')).dt.days

test_data['used_days'] = (pd.to_datetime(test_data['creatDate'], format='%Y%m%d', errors='coerce') - 
                            pd.to_datetime(test_data['regDate'], format='%Y%m%d', errors='coerce')).dt.days

train_data['used_year'] = train_data['used_days'] / 365

test_data['used_year'] = test_data['used_days'] / 365

In [ ]:
##2.2. build some discrete features from numeric ones using bins.
train_data['power'] = np.clip(train_data.power, 0, 600)
test_data['power'] = np.clip(test_data.power, 0, 600)

ax = train_data.power.hist(bins=50)
ax.set_xlabel('power')
ax.set_title('power distribution')

In [ ]:
power_bin = [i*10 for i in range(31)]
train_data['power_bin'] = pd.cut(train_data.power, bins=power_bin, labels=False)
test_data['power_bin'] = pd.cut(test_data.power, bins=power_bin, labels=False)

In [ ]:
ax = train_data.model.hist(bins=30)
ax.set_xlabel('model')
ax.set_title('model distribution')

In [ ]:
model_bin = [i*10 for i in range(14)]
train_data['model_bin'] = pd.cut(train_data.model, bins=model_bin, labels=False)
test_data['model_bin'] = pd.cut(test_data.model, bins=model_bin, labels=False)

In [ ]:
##2.3. build some cross-interaction features
def cross_interaction(train_data, cat_feature, num_feature, test_data):
    column_name = {'max':cat_feature+'_'+num_feature+'_max', 
              'median':cat_feature+'_'+num_feature+'_median', 
              'min':cat_feature+'_'+num_feature+'_min', 
              'sum':cat_feature+'_'+num_feature+'_sum', 
              'std':cat_feature+'_'+num_feature+'_std', 
              'mean':cat_feature+'_'+num_feature+'_mean'}
    cros_feature = train_data.groupby(cat_feature)[num_feature].agg(['max', 'median', 'min', 'sum', 'std', 'mean'])\
                                                            .reset_index().rename(columns=column_name)
    train_data = train_data.merge(cros_feature, how='left', on=cat_feature)
    test_data = test_data.merge(cros_feature, how='left', on=cat_feature)
    
    return train_data, test_data
    
train_data, test_data = cross_interaction(train_data, 'brand', 'price', test_data)
train_data, test_data = cross_interaction(train_data, 'bodyType', 'price', test_data)


In [ ]:
## 2.4. drop non-significant features
features_to_drop = ['name', 'regDate', 'regionCode', 'seller', 'offerType', 'creatDate']

train_data = train_data.drop(columns=features_to_drop)
test_data = test_data.drop(columns=features_to_drop)

In [ ]:
## 2.5 transfer the target variable
from sklearn.preprocessing import PowerTransformer
power_trans = PowerTransformer(method='box-cox')
train_data['price_trasfer'] = power_trans.fit_transform(train_data.loc[:,['price']])

In [ ]:
ax = sns.distplot(train_data.price_trasfer)
ax.set_title('transfered price variable')

## 3. Modeling
- additional analyses will be performed in the future to imporve predictive performance

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb

### 3.1 XGBoost

In [ ]:
train_data.notRepairedDamage = train_data.notRepairedDamage.astype(float)
test_data.notRepairedDamage = test_data.notRepairedDamage.astype(float)

xgb_model = xgb.XGBRegressor(n_estimators=1500, learning_rate=0.05)
scores = cross_val_score(xgb_model, X=train_data.drop(columns=['SaleID', 'price', 'price_trasfer']), y=train_data.price, cv=5, scoring=make_scorer(mean_absolute_error))
print(scores)

In [ ]:
xgb_model = xgb.XGBRFRegressor(n_estimators=1500, learning_rate=0.1)
xgb_model.fit(X=train_data.drop(columns=['SaleID', 'price', 'price_trasfer']), y=train_data.price)

pred_price = xgb_model.predict(data=test_data.drop(columns='SaleID'))
prediction_price = test_data.loc[:, ['SaleID']]
prediction_price['price'] = pred_price

prediction_price.to_csv('used_car_testB_submit_xgb.csv', index=False)

### 3.2 LightGBM

In [ ]:
lgb_model = lgb.LGBMRegressor(n_estimators=1500, learning_rate=0.05)
scores = cross_val_score(lgb_model, X=train_data.drop(columns=['SaleID', 'price', 'price_trasfer']), y=train_data.price, cv=5, scoring=make_scorer(mean_absolute_error))
print(scores)

In [ ]:
lgb_model = lgb.LGBMRegressor(n_estimators=1500, learning_rate=0.05)
lgb_model.fit(X=train_data.drop(columns=['SaleID', 'price', 'price_trasfer']), y=train_data.price_trasfer)

pred_price = lgb_model.predict(X=test_data.drop(columns='SaleID'))
prediction_price = test_data.loc[:, ['SaleID']]
prediction_price['price'] = power_trans.inverse_transform(pred_price.reshape(-1, 1))

prediction_price.to_csv('used_car_testB_submit_lgb.csv', index=False)

### 3.3 random forest

In [ ]:
train_data_no_missing  = train_data.dropna()
test_data_no_missing = test_data.dropna()
rf = RandomForestRegressor()
rf.fit(train_data_no_missing.drop(columns=['SaleID', 'price', 'price_trasfer']), y=train_data_no_missing.price)

pred_price = rf.predict(X=test_data_no_missing.drop(columns='SaleID'))
prediction_price = test_data_no_missing.loc[:, ['SaleID']]
prediction_price['price'] = pred_price

prediction_price.to_csv('used_car_testB_submit_rf.csv', index=False)

### 3.4 logistic regression

In [ ]:
lg = LogisticRegression()
lg.fit(train_data_no_missing.drop(columns=['SaleID', 'price', 'price_trasfer']), y=train_data_no_missing.price)

pred_price = lg.predict(X=test_data_no_missing.drop(columns='SaleID'))
prediction_price = test_data_no_missing.loc[:, ['SaleID']]
prediction_price['price'] = pred_price

prediction_price.to_csv('used_car_testB_submit_lg.csv', index=False)